In [43]:
# Dependencies and Setup
import os

import pandas as pd
import numpy as np
import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

from flask import Flask, jsonify, render_template
#from flask_sqlalchemy import SQLAlchemy

# Datetime
import datetime as dt


# Extract Data: From .csv files into Pandas DataFrames

In [44]:
# Files to Load
# calendar_file_path = "static/data/calendar.csv"
listings_details_file_path = "static/data/listings_details.csv"
# reviews_details_file_path = "static/data/reviews_details.csv"

In [46]:
# Read file and store into Pandas data frames
# calendar_df = pd.read_csv(calendar_file_path)
listings_details_df = pd.read_csv(listings_details_file_path)
# reviews_details_df = pd.read_csv(reviews_details_file_path)

# Print the listing_details DataFrame (which has the room type data values)
listings_details_df.head()
# 2387 rows x 106 columns

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,38585,https://www.airbnb.com/rooms/38585,2.020050e+13,6/4/2020,Charming Victorian home - twin beds + breakfast,Per the City Council of Asheville. Due to the ...,"Charming room with 2 twin size beds, furnished...",Per the City Council of Asheville. Due to the ...,none,Our North Asheville neighborhood stretches alo...,...,f,f,moderate,t,t,1,0,1,0,1.18
1,80905,https://www.airbnb.com/rooms/80905,2.020050e+13,6/4/2020,French Chic Loft,Let yourself melt into the delectable décor of...,Have you ever gazed at dreamy photos in a maga...,Let yourself melt into the delectable décor of...,none,"Quiet, tucked away from the hustle and bustle ...",...,t,f,super_strict_60,f,f,11,11,0,0,1.01
2,108061,https://www.airbnb.com/rooms/108061,2.020050e+13,6/4/2020,Walk to stores/parks/downtown. Fenced yard/Pet...,Walk to town in ten minutes! Monthly rental in...,True Asheville...artist styled apartment with ...,Walk to town in ten minutes! Monthly rental in...,none,"I love my neighborhood! Its friendly, easy-goi...",...,f,f,strict_14_with_grace_period,f,f,2,2,0,0,0.84
3,155305,https://www.airbnb.com/rooms/155305,2.020050e+13,5/29/2020,Cottage! BonPaul + Sharky's Hostel,NaN,Private cottage located behind the main house ...,Private cottage located behind the main house ...,none,"We are within easy walk of pubs, breweries, mu...",...,t,f,moderate,f,f,7,1,2,4,2.23
4,156805,https://www.airbnb.com/rooms/156805,2.020050e+13,5/29/2020,"Private Room ""Ader"" at BPS Hostel",NaN,Private Rooms at Bon Paul and Sharky's Hostel....,Private Rooms at Bon Paul and Sharky's Hostel....,none,"Easy walk to pubs, cafes, bakery, breweries, l...",...,t,f,moderate,f,f,7,1,2,4,0.63


# Transform Data: To breakdown by Room Type

In [47]:
# Review the columns available for use - Looking for room type
list(listings_details_df)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


In [48]:
# Filtering on columns of interest 
filtered_listings_details_df = listings_details_df.filter(["id", "name",
                                                           "latitude", "longitude",
                                                           "neighbourhood_cleansed","city",  
                                                           "property_type", "room_type",
                                                           'accomodates', "bedrooms", "bathrooms",
                                                           'price', "host_since", "host_is_superhost",
                                                           "guests_included", "minimum_nights", 
                                                           "number_of_reviews", "review_scores_rating",
                                                          "instant_bookable", "cancellation_policy"])
filtered_listings_details_df

# 2387 rows x 19 columns

,id,name,latitude,longitude,neighbourhood_cleansed,city,property_type,room_type,bedrooms,bathrooms,price,host_since,host_is_superhost,guests_included,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,cancellation_policy
0,38585,Charming Victorian home - twin beds + breakfast,35.65146,-82.62792,28804,Asheville,House,Private room,1,1.0,$50.00,7/13/2010,t,1,1,138,96.0,f,moderate
1,80905,French Chic Loft,35.59779,-82.55540,28801,Asheville,Loft,Entire home/apt,3,2.0,$765.00,3/7/2011,t,6,1,108,96.0,t,super_strict_60
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,35.60670,-82.55563,28801,Asheville,Apartment,Entire home/apt,1,1.0,$75.00,12/16/2010,f,2,30,89,90.0,f,strict_14_with_grace_period
3,155305,Cottage! BonPaul + Sharky's Hostel,35.57864,-82.59578,28806,Asheville,Guesthouse,Entire home/apt,1,1.0,$107.00,6/26/2011,t,2,1,240,90.0,t,moderate
4,156805,"Private Room ""Ader"" at BPS Hostel",35.57864,-82.59578,28806,Asheville,House,Private room,1,2.5,$71.00,6/26/2011,t,2,1,67,90.0,t,moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,43473737,Peace of Paradise,35.59714,-82.46050,28805,Asheville,Apartment,Entire home/apt,1,1.0,$65.00,9/10/2015,f,1,1,4,85.0,f,moderate
2383,43495845,Luxury Downtown Asheville Condo with Mountain ...,35.59451,-82.55317,28801,Asheville,Apartment,Entire home/apt,1,1.0,$305.00,4/8/2020,f,2,1,0,NaN,t,strict_14_with_grace_period
2384,43509635,Monthly furnished rental - Walk to town !,35.60995,-82.55054,28801,Asheville,House,Entire home/apt,3,2.0,$155.00,5/21/2020,f,1,30,0,NaN,t,strict_14_with_grace_period
2385,43525648,"Close to Downtown, Secluded, Hot Tub, WiFi, Pe...",35.49753,-82.50975,28803,Asheville,Cabin,Entire home/apt,2,2.0,$225.00,2/17/2012,f,1,4,0,NaN,t,strict_14_with_grace_period


In [49]:
# Price is in string format - Remove '$'' and ',''  and convert to float
filtered_listings_details_df.price = filtered_listings_details_df.price.replace('[\$,]', '', regex=True).astype(float)
filtered_listings_details_df

,id,name,latitude,longitude,neighbourhood_cleansed,city,property_type,room_type,bedrooms,bathrooms,price,host_since,host_is_superhost,guests_included,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,cancellation_policy
0,38585,Charming Victorian home - twin beds + breakfast,35.65146,-82.62792,28804,Asheville,House,Private room,1,1.0,50.0,7/13/2010,t,1,1,138,96.0,f,moderate
1,80905,French Chic Loft,35.59779,-82.55540,28801,Asheville,Loft,Entire home/apt,3,2.0,765.0,3/7/2011,t,6,1,108,96.0,t,super_strict_60
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,35.60670,-82.55563,28801,Asheville,Apartment,Entire home/apt,1,1.0,75.0,12/16/2010,f,2,30,89,90.0,f,strict_14_with_grace_period
3,155305,Cottage! BonPaul + Sharky's Hostel,35.57864,-82.59578,28806,Asheville,Guesthouse,Entire home/apt,1,1.0,107.0,6/26/2011,t,2,1,240,90.0,t,moderate
4,156805,"Private Room ""Ader"" at BPS Hostel",35.57864,-82.59578,28806,Asheville,House,Private room,1,2.5,71.0,6/26/2011,t,2,1,67,90.0,t,moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,43473737,Peace of Paradise,35.59714,-82.46050,28805,Asheville,Apartment,Entire home/apt,1,1.0,65.0,9/10/2015,f,1,1,4,85.0,f,moderate
2383,43495845,Luxury Downtown Asheville Condo with Mountain ...,35.59451,-82.55317,28801,Asheville,Apartment,Entire home/apt,1,1.0,305.0,4/8/2020,f,2,1,0,NaN,t,strict_14_with_grace_period
2384,43509635,Monthly furnished rental - Walk to town !,35.60995,-82.55054,28801,Asheville,House,Entire home/apt,3,2.0,155.0,5/21/2020,f,1,30,0,NaN,t,strict_14_with_grace_period
2385,43525648,"Close to Downtown, Secluded, Hot Tub, WiFi, Pe...",35.49753,-82.50975,28803,Asheville,Cabin,Entire home/apt,2,2.0,225.0,2/17/2012,f,1,4,0,NaN,t,strict_14_with_grace_period


In [50]:
# Convert host_since from string to datetime
filtered_listings_details_df['host_since'] = pd.to_datetime(filtered_listings_details_df['host_since'])
filtered_listings_details_df

,id,name,latitude,longitude,neighbourhood_cleansed,city,property_type,room_type,bedrooms,bathrooms,price,host_since,host_is_superhost,guests_included,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,cancellation_policy
0,38585,Charming Victorian home - twin beds + breakfast,35.65146,-82.62792,28804,Asheville,House,Private room,1,1.0,50.0,2010-07-13,t,1,1,138,96.0,f,moderate
1,80905,French Chic Loft,35.59779,-82.55540,28801,Asheville,Loft,Entire home/apt,3,2.0,765.0,2011-03-07,t,6,1,108,96.0,t,super_strict_60
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,35.60670,-82.55563,28801,Asheville,Apartment,Entire home/apt,1,1.0,75.0,2010-12-16,f,2,30,89,90.0,f,strict_14_with_grace_period
3,155305,Cottage! BonPaul + Sharky's Hostel,35.57864,-82.59578,28806,Asheville,Guesthouse,Entire home/apt,1,1.0,107.0,2011-06-26,t,2,1,240,90.0,t,moderate
4,156805,"Private Room ""Ader"" at BPS Hostel",35.57864,-82.59578,28806,Asheville,House,Private room,1,2.5,71.0,2011-06-26,t,2,1,67,90.0,t,moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,43473737,Peace of Paradise,35.59714,-82.46050,28805,Asheville,Apartment,Entire home/apt,1,1.0,65.0,2015-09-10,f,1,1,4,85.0,f,moderate
2383,43495845,Luxury Downtown Asheville Condo with Mountain ...,35.59451,-82.55317,28801,Asheville,Apartment,Entire home/apt,1,1.0,305.0,2020-04-08,f,2,1,0,NaN,t,strict_14_with_grace_period
2384,43509635,Monthly furnished rental - Walk to town !,35.60995,-82.55054,28801,Asheville,House,Entire home/apt,3,2.0,155.0,2020-05-21,f,1,30,0,NaN,t,strict_14_with_grace_period
2385,43525648,"Close to Downtown, Secluded, Hot Tub, WiFi, Pe...",35.49753,-82.50975,28803,Asheville,Cabin,Entire home/apt,2,2.0,225.0,2012-02-17,f,1,4,0,NaN,t,strict_14_with_grace_period


In [51]:
# Remove rows with missing data in any column 2171 rows x 19 columns
clean_listings_details_df = filtered_listings_details_df.dropna(how='any')
clean_listings_details_df

# Do not do this on the whole dataset

,id,name,latitude,longitude,neighbourhood_cleansed,city,property_type,room_type,bedrooms,bathrooms,price,host_since,host_is_superhost,guests_included,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,cancellation_policy
0,38585,Charming Victorian home - twin beds + breakfast,35.65146,-82.62792,28804,Asheville,House,Private room,1,1.0,50.0,2010-07-13,t,1,1,138,96.0,f,moderate
1,80905,French Chic Loft,35.59779,-82.55540,28801,Asheville,Loft,Entire home/apt,3,2.0,765.0,2011-03-07,t,6,1,108,96.0,t,super_strict_60
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,35.60670,-82.55563,28801,Asheville,Apartment,Entire home/apt,1,1.0,75.0,2010-12-16,f,2,30,89,90.0,f,strict_14_with_grace_period
3,155305,Cottage! BonPaul + Sharky's Hostel,35.57864,-82.59578,28806,Asheville,Guesthouse,Entire home/apt,1,1.0,107.0,2011-06-26,t,2,1,240,90.0,t,moderate
4,156805,"Private Room ""Ader"" at BPS Hostel",35.57864,-82.59578,28806,Asheville,House,Private room,1,2.5,71.0,2011-06-26,t,2,1,67,90.0,t,moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,43302001,Asheville Hidden Acres,35.54624,-82.63113,28806,Asheville,Guesthouse,Entire home/apt,2,1.0,75.0,2020-04-27,f,1,1,4,100.0,t,moderate
2374,43332564,Asheville Glamping in the city with a Hot tub,35.59283,-82.59437,28806,Asheville,Bus,Entire home/apt,1,1.0,129.0,2012-09-24,f,1,1,4,100.0,t,strict_14_with_grace_period
2377,43403464,"Brand New, Immaculate Sanctuary in North Ashev...",35.62865,-82.56802,28804,Asheville,Apartment,Entire home/apt,1,1.0,99.0,2015-09-25,f,1,2,1,100.0,f,flexible
2378,43412676,Asheville Area Guest Quarters w/Private Pool,35.46087,-82.47091,28732,Fletcher,Guest suite,Entire home/apt,2,1.0,125.0,2015-09-21,f,2,2,2,100.0,t,moderate


In [52]:
# Calculate total number of "unique" listings by using function "value counts"
listing_count = len(clean_listings_details_df["id"].value_counts())
listing_count

2171

In [53]:
# Create a list of top 5 neighbourhoords in terms of listings
top5_neighbourhoods_by_listings = clean_listings_details_df['neighbourhood_cleansed'].value_counts().iloc[:5].index.tolist()
top5_neighbourhoods_by_listings

[28806, 28801, 28803, 28804, 28805]

In [111]:
# Create a dataframe to group neighbourhoods by average room price
neighbourhood_avgprice = pd.DataFrame(clean_listings_details_df.groupby(['neighbourhood_cleansed']).price.mean().reset_index())
neighbourhood_avgprice.round({'price': 2})


,neighbourhood_cleansed,price
0,28704,164.61
1,28715,123.67
2,28732,157.20
3,28801,190.39
4,28803,148.30
5,28804,153.09
6,28805,130.91
7,28806,116.49


In [112]:
# Create a dataframe to filter highest priced neighbourhoods - Top 5
highest_priced_neighbourhoods = neighbourhood_avgprice[neighbourhood_avgprice['neighbourhood_cleansed'].isin(top5_neighbourhoods_by_listings)].sort_values('price',ascending=False)
highest_priced_neighbourhoods.round({'price': 2})

,neighbourhood_cleansed,price
3,28801,190.39
5,28804,153.09
4,28803,148.30
6,28805,130.91
7,28806,116.49


In [113]:
highest_priced_neighbourhoods.rename(columns={"neighbourhood_cleansed": "neighbourhood", "price": "avgprice"}, inplace = True)
highest_priced_neighbourhoods.round({'avgprice': 2})

,neighbourhood,avgprice
3,28801,190.39
5,28804,153.09
4,28803,148.30
6,28805,130.91
7,28806,116.49


### Listings by Room Type 

In [76]:
# Listings grouped by Room Type
listings_by_roomtype = clean_listings_details_df.groupby("room_type")["id"].nunique()
listings_by_roomtype = listings_by_roomtype.sort_values(ascending = False)

In [77]:
# Create dataframe and reset index
listings_by_roomtype_df = pd.DataFrame(listings_by_roomtype)
listings_by_roomtype_df = listings_by_roomtype_df.reset_index()
listings_by_roomtype_df

,room_type,id
0,Entire home/apt,1683
1,Private room,468
2,Hotel room,11
3,Shared room,9


In [78]:
listings_by_roomtype_df.rename(columns={"room_type": "roomtype", "id": "listings"}, inplace = True)
listings_by_roomtype_df

,roomtype,listings
0,Entire home/apt,1683
1,Private room,468
2,Hotel room,11
3,Shared room,9


### Listings by Property Type

In [79]:
# Listings grouped by Property Type
listings_by_propertytype = clean_listings_details_df.groupby("property_type")["id"].nunique()
listings_by_propertytype = listings_by_propertytype.sort_values(ascending = False)

In [80]:
# Create dataframe and reset index
listings_by_propertytype_df = pd.DataFrame(listings_by_propertytype)
listings_by_propertytype_df = listings_by_propertytype_df.reset_index()
listings_by_propertytype_df

,property_type,id
0,House,903
1,Apartment,355
2,Guest suite,327
3,Bungalow,109
4,Condominium,92
5,Guesthouse,92
6,Cottage,69
7,Cabin,57
8,Townhouse,35
9,Loft,30


In [81]:
listings_by_propertytype_df.rename(columns={"property_type": "propertytype", "id": "listings"}, inplace = True)
listings_by_propertytype_df

,propertytype,listings
0,House,903
1,Apartment,355
2,Guest suite,327
3,Bungalow,109
4,Condominium,92
5,Guesthouse,92
6,Cottage,69
7,Cabin,57
8,Townhouse,35
9,Loft,30


## Top 10 Types of Properties based on Listings

In [82]:
# Top 10 Types of Properties based on number of listings 
top_propertytypes = listings_by_propertytype_df.head(10)
top_propertytypes

,propertytype,listings
0,House,903
1,Apartment,355
2,Guest suite,327
3,Bungalow,109
4,Condominium,92
5,Guesthouse,92
6,Cottage,69
7,Cabin,57
8,Townhouse,35
9,Loft,30


## Data Prep for Chloropeth of Neighbourhoods (No. of listings, Avg. Price & Avg. Rating)

In [83]:
# Create new dataframe for listings grouped by neighbourhood
listings = pd.DataFrame(clean_listings_details_df.groupby(['neighbourhood_cleansed']).id.count()).reset_index()

# Rename column in neighbourhood_cleansed dataframe
listings.rename(columns={'neighbourhood_cleansed':'neighbourhood','id':'listings'},inplace=True)
listings

,neighbourhood,listings
0,28704,120
1,28715,43
2,28732,45
3,28801,502
4,28803,310
5,28804,266
6,28805,255
7,28806,630


In [114]:
# Create new dataframe for listings grouped by neighbourhood
price = pd.DataFrame(clean_listings_details_df.groupby(['neighbourhood_cleansed']).price.mean()).reset_index()

# Rename column in neighbourhood_cleansed dataframe
price.rename(columns={'neighbourhood_cleansed':'neighbourhood','price':'avgprice'},inplace=True)
price.round({'avgprice': 2})

,neighbourhood,avgprice
0,28704,164.61
1,28715,123.67
2,28732,157.20
3,28801,190.39
4,28803,148.30
5,28804,153.09
6,28805,130.91
7,28806,116.49


In [115]:
# Create a new dataframe for ratings grouped by neighbourhood
ratings = pd.DataFrame(clean_listings_details_df.groupby(['neighbourhood_cleansed']).review_scores_rating.mean()).reset_index()

# Rename column in neighbourhood_cleansed dataframe
ratings.rename(columns={'neighbourhood_cleansed':'neighbourhood','review_scores_rating':'avgrating'}, inplace=True)
ratings.round({'avgrating': 2})

,neighbourhood,avgrating
0,28704,97.92
1,28715,97.60
2,28732,97.84
3,28801,97.38
4,28803,98.18
5,28804,97.60
6,28805,98.03
7,28806,97.57


In [116]:
# Merge listings, price and ratings
listings_by_neighbourhood = listings.merge(price, on='neighbourhood')
listings_by_neighbourhood = listings_by_neighbourhood.merge(ratings, on='neighbourhood')
listings_by_neighbourhood.round({'avgprice': 2, 'avgrating': 2})

,neighbourhood,listings,avgprice,avgrating
0,28704,120,164.61,97.92
1,28715,43,123.67,97.60
2,28732,45,157.20,97.84
3,28801,502,190.39,97.38
4,28803,310,148.30,98.18
5,28804,266,153.09,97.60
6,28805,255,130.91,98.03
7,28806,630,116.49,97.57


## Create CSV files to use in plotting

In [117]:
clean_listings_details_df.to_csv(r'static/data/listings.csv', index=False, header=True)

# For bar plot of listings by Property/Room Type per neighbourhood
listings_by_roomtype_df.to_csv(r'static/data/listings_by_roomtype.csv', index=False, header=True)

listings_by_propertytype_df.to_csv(r'static/data/listings_by_propertytype.csv', index=False, header=True)

top_propertytypes.to_csv(r'static/data/top_propertytypes.csv', index=False, header=True)

# For chloropethplot of highest priced neighbourhoods
highest_priced_neighbourhoods.to_csv(r'static/data/highest_priced_neighbourhoods.csv', index=False, header=True)

# For Chloropeth of the following:
# 1) Number of Listings per neighbourhood
# 2) Avg Price of airbnb listings per neighbourhood 
# 3) Avg Rating of airbnb listings per neighbourhood and 3)

listings_by_neighbourhood.to_csv(r'static/data/listings_by_neighbourhood.csv', index=False, header=True)

# Load Data into the PostgreSQL Database

In [118]:
# Dependencies
from sqlalchemy import create_engine
from config import (postgres_id, postgres_pw)

In [119]:
# Connect to a local database
conn_str = f"{postgres_id}:{postgres_pw}@localhost:5432/airbnb_db"
engine = create_engine(f'postgresql://{conn_str}')

In [120]:
# Reflect the database 
Base = automap_base()

# Check table names in the database
engine.table_names()

['listings_by_propertytype',
 'top_propertytypes',
 'listings',
 'listings_by_roomtype',
 'highest_priced_neighbourhoods',
 'listings_by_neighbourhood']

## Load the DataFrames into the PostgreSQL Database

In [36]:
# Original Files
# calendar_df.to_sql(name='calendar',  if_exists='append', con=engine,index=False)
# listings_details_df.to_sql(name='listings_details',  if_exists='append', con=engine,index=False)
# reviews_details_df.to_sql(name='reviews_details',  if_exists='append', con=engine,index=False)

In [121]:
clean_listings_details_df.to_sql(name='listings',  if_exists='append', con=engine,index=False)

In [122]:
# Confirm that the data loaded correctly - 2387 rows
pd.read_sql_query('select * from listings', con=engine)

,id,name,latitude,longitude,neighbourhood_cleansed,city,property_type,room_type,accomodates,bedrooms,bathrooms,price,host_since,host_is_superhost,guests_included,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,cancellation_policy
0,38585,Charming Victorian home - twin beds + breakfast,36,-83,28804,Asheville,House,Private room,None,1,1,50.0,2010-07-13,True,1,1,138,96,False,moderate
1,80905,French Chic Loft,36,-83,28801,Asheville,Loft,Entire home/apt,None,3,2,765.0,2011-03-07,True,6,1,108,96,True,super_strict_60
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,36,-83,28801,Asheville,Apartment,Entire home/apt,None,1,1,75.0,2010-12-16,False,2,30,89,90,False,strict_14_with_grace_period
3,155305,Cottage! BonPaul + Sharky's Hostel,36,-83,28806,Asheville,Guesthouse,Entire home/apt,None,1,1,107.0,2011-06-26,True,2,1,240,90,True,moderate
4,156805,"Private Room ""Ader"" at BPS Hostel",36,-83,28806,Asheville,House,Private room,None,1,3,71.0,2011-06-26,True,2,1,67,90,True,moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166,43302001,Asheville Hidden Acres,36,-83,28806,Asheville,Guesthouse,Entire home/apt,None,2,1,75.0,2020-04-27,False,1,1,4,100,True,moderate
2167,43332564,Asheville Glamping in the city with a Hot tub,36,-83,28806,Asheville,Bus,Entire home/apt,None,1,1,129.0,2012-09-24,False,1,1,4,100,True,strict_14_with_grace_period
2168,43403464,"Brand New, Immaculate Sanctuary in North Ashev...",36,-83,28804,Asheville,Apartment,Entire home/apt,None,1,1,99.0,2015-09-25,False,1,2,1,100,False,flexible
2169,43412676,Asheville Area Guest Quarters w/Private Pool,35,-82,28732,Fletcher,Guest suite,Entire home/apt,None,2,1,125.0,2015-09-21,False,2,2,2,100,True,moderate


In [123]:
listings_by_roomtype_df.to_sql(name='listings_by_roomtype',  if_exists='append', con=engine,index=False)

In [124]:
# Confirm that the data loaded correctly 
pd.read_sql_query('select * from listings_by_roomtype', con=engine)

,roomtype,listings
0,Entire home/apt,1683
1,Private room,468
2,Hotel room,11
3,Shared room,9


In [125]:
listings_by_propertytype_df.to_sql(name='listings_by_propertytype',  if_exists='append', con=engine,index=False)

In [126]:
# Confirm that the data loaded correctly -  rows
pd.read_sql_query('select * from listings_by_propertytype', con=engine)

,propertytype,listings
0,House,903
1,Apartment,355
2,Guest suite,327
3,Bungalow,109
4,Condominium,92
5,Guesthouse,92
6,Cottage,69
7,Cabin,57
8,Townhouse,35
9,Loft,30


In [127]:
top_propertytypes.to_sql(name='top_propertytypes',  if_exists='append', con=engine,index=False)

In [128]:
# Confirm that the data loaded correctly 
pd.read_sql_query('select * from top_propertytypes', con=engine)

,propertytype,listings
0,House,903
1,Apartment,355
2,Guest suite,327
3,Bungalow,109
4,Condominium,92
5,Guesthouse,92
6,Cottage,69
7,Cabin,57
8,Townhouse,35
9,Loft,30


In [129]:
highest_priced_neighbourhoods.to_sql(name='highest_priced_neighbourhoods',  if_exists='append', con=engine,index=False)

In [130]:
# Confirm that the data loaded correctly 
pd.read_sql_query('select * from highest_priced_neighbourhoods', con=engine)

,neighbourhood,avgprice
0,28801,190.386454
1,28804,153.086466
2,28803,148.303226
3,28805,130.909804
4,28806,116.490476


In [131]:
listings_by_neighbourhood.to_sql(name='listings_by_neighbourhood',  if_exists='append', con=engine,index=False)

In [132]:
# Confirm that the data loaded correctly 
pd.read_sql_query('select * from listings_by_neighbourhood', con=engine)

,neighbourhood,listings,avgprice,avgrating
0,28704,120,164.608333,97.916667
1,28715,43,123.674419,97.604651
2,28732,45,157.200000,97.844444
3,28801,502,190.386454,97.382470
4,28803,310,148.303226,98.180645
5,28804,266,153.086466,97.601504
6,28805,255,130.909804,98.031373
7,28806,630,116.490476,97.568254
